# Dynamic Programming Notes

Dynamic programming is a concept of solving recurdsive problems. There are two possible approches: "push" (BFS-like aproach) and "pull" (DFS-like approach).

In [1]:
import numpy as np

### First example

For a given $k, n\in\mathbb{N}$ let's try to compute ${n \choose k}$. For that we will need to use the Pascal's triangle identity

\begin{align}
{n \choose k} = {n - 1 \choose k - 1} + {n - 1 \choose k}.
\tag{Pascal's triangle identity}
\end{align}

To prove the above, we can see that on the left hand side we are choosing $k$ elements from the $n$-element set. So lets pick any element from our $n$-element set. If we want to include this element into our $k$-element subset, we need to choose another $k-1$ elements from remaining $n-1$ elements. If we do not want to inclide that element, we need to select $k$ elements from the remaining $n-1$ elements.

Now, let $p(n, k) = {n \choose k}$. We can write

\begin{equation*}
\begin{aligned}
&p(n, 0) = 0, \\
&p(n, k) = 0 &\hbox{ for } n < k, \\
&p(n, k) = p(n - 1, k - 1) + p(n - 1, k) &\hbox{ for } n \ge k > 0.
\end{aligned}
\end{equation*}

From above formula it is clear, that it is possible to compute $p(n, k)$ for any $n, k\in\mathbb{N}$ using $O(n\times k)$ memory and
$O(n\times k)$ time.

In [2]:
def n_choose_k(N: int, K: int) -> int:
    p = np.zeros((N+1, K+1)).astype(int)
    p[:, 0] = 1
    for k in range(1, K + 1):
        for n in range(k, N + 1):
            p[n, k] = p[n-1, k-1] + p[n-1, k]
    
    return p[N, K]

Let's write a benchmark method (for tests), which is the most efficient way to compute $n \choose k$.

In [18]:
def n_choose_k_benchmark(N: int, K: int) -> int:
    if K > N:
        return 0
    
    result = 1
    for i in range(1, K + 1):
        if result % i == 0:
            result = result // i
            result = result * (N - i + 1) 
        else:
            result = result * (N - i + 1) 
            result = result // i
    
    return result

In [19]:
for N in range(60):
    for K in range(N + 1):
        assert n_choose_k(N, K) == n_choose_k_benchmark(N, K), f'error for {N=}, {K=}'

We can see, that the funcion is working correctly, however, we are using a lot of memory. Let's try to optimize the memory usage. 

First observation is that the values in a given row of $p$ depend only on values from a previous row. Let's change our function a little bit to optimize the memory usage to be $O(n)$.

Note: function after the proposed modification will need to perform more operation, but the time complexity will still be $O(n\times k)$.

In [5]:
def n_choose_k_mem_opt(N: int, K: int) -> int:
    p_prev = np.ones(N+1).astype(int)
    p_next = np.zeros(N+1).astype(int)
    for k in range(1, K + 1):
        for n in range(N + 1):
            if n < k:
                p_next[n] = 0
            else:
                p_next[n] = p_prev[n-1] + p_next[n-1]
        p_next, p_prev = p_prev, p_next
    
    return p_prev[N]


In [20]:
for N in range(60):
    for K in range(N + 1):
        assert n_choose_k_mem_opt(N, K) == n_choose_k_benchmark(N, K), f'error for {N=}, {K=}'


### More complex example

Let's consider the first (trivial) example. For a given set of positive numbers $S = \{s_1, s_2, \ldots, s_m\}$ and a given number $N$ we need to calculate the number of ways to write a given number $N$ as a sum of the numbers from $S$ (each number can be used multiple times). For example, 
for $S=\{1, 2\}$ and $N=3$ we have three ways:

* $1 + 1 + 1$
* $2 + 1$
* $1 + 2$

**Solution (pull).** Let $Q(k)$ be the set of all sequences of numbers from $S$, that sum to $k$. For any combination $p$, let $p^{\prime}$ denote the combination obtained from $p$ by removing it's last element and let $p_i$ to be the $i$-th element of $p$. We can see, that $\#Q(0) = 1$, as only empty sum equals zero. To compute the value of $\#Q(k)$ we need to compute the values for $\#Q(s)$ for each $s\in S(k)$, where $S(k) = \{x\colon (\exists s\in S) (x + s = k)\}$. Assuming we already computed them, we can express $\# Q(k)$ as

\begin{align}
\#Q(k) = \sum_{x\in S(k)} \# Q(x).
\tag{pull formula}
\end{align}

In that way we are taking all the possible combinations of elements from $S$ that sum up to $k$. 

To see this, let $p$ be a sequence from $Q(k)$ of length $j$. We can see, that $p^{\prime}$ sums up to $k-p_j$. Of course $k-p_j\in S(k)$ and therefore $p^{\prime}\in Q(k-p_j)$. From that we conclude, that $p$ (as a unique extension of $p^{\prime}$) is counted in the $\# Q(x)$ term of the sum above (for $x = k-p_j$). Conversly, take any two combinations $p, q$ from $Q(k)$ of length $t, u$ (respectively). If $p^{\prime} = q^{\prime}$ then $p_t = q_u = q_t$, because

$$
\sum_{i=1}^t p_i = q_t + \sum_{i=1}^{t-1} p_i = q_t + \sum_{j=1}^u q_j,
$$

so for each $x\in S(k)$ each sequence $r\in Q(x)$ is mapped to at most one sequence $\overline{r}\in Q(k)$.

**Solution (push).** Let $P(k, l)$ will be a familly of sequences, where $0\le k\le N$ and $0\le l\le m$. The definition of $P(k, l)$ is as follows
\begin{equation}
\begin{aligned}
& P(0, 0) = \{\emptyset\} \\
& P(0, k) = \emptyset, \hbox{ for } 1 \le k \le N \\
& P(k, l) = P(k, l-1)\cup \{p*s_k(p)\colon p\in P(k-1, l-1) \wedge s_k(p)\in S\},
\end{aligned}
\end{equation}
where $s_k(p) = k - \sum_i p_i$ and $*$ is a concatenation operator.

**Exercise**: Prove, that $\# Q(N) = \#P(N, m)$.

### Implementation

In [7]:
def n_sequences_pull(S: set[int], N: int) -> int:
    Q = np.zeros(N + 1).astype(int)
    Q[0] = 1
    for k in range(1, N + 1):
        Q[k] += sum(Q[k - s] for s in S if k - s >= 0)
    
    return Q[N]
            

In [8]:
def n_sequences_push(S: set[int], N: int) -> int:
    P = np.zeros(N + 1).astype(int)
    P[0] = 1
    for k in range(N+1):
        for s in S:
            if k + s <= N:
                P[k + s] += P[k]

    return P[N]

In [9]:
n_sequences_pull(S=set(range(1, 100)), N=4)

8

In [10]:
n_sequences_push(S=set(range(1, 100)), N=4)

8